In [15]:
from fractions import Fraction
from typing import Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [16]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present (though your files use decimals).
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)


In [17]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

# Phân tách LU


## Thuật toán

Ax = B

- Viết A = LU --> LUx = B
- Đặt y=Ux --> Ly=B --> y = L^-1 B
và x=U^-1y

In [18]:
def lu_decomposition(A):
    """
    Perform LU decomposition on matrix A with U having unit diagonal.
    
    Parameters:
    A (numpy.ndarray): Input square matrix
    
    Returns:
    tuple: (L, U) matrices where U has diagonal elements = 1
    """
    n = len(A)
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    
    # Set diagonal of U to 1
    np.fill_diagonal(U, 1)
    
    # LU decomposition
    for i in range(n):
        # Lower triangular matrix
        L[i][i] = A[i][i] - sum(L[i][j] * U[j][i] for j in range(i))
        
        for k in range(i + 1, n):
            # Lower triangular elements
            L[k][i] = A[k][i] - sum(L[k][j] * U[j][i] for j in range(i))
            
            # Upper triangular elements (normalized)
            U[i][k] = (A[i][k] - sum(L[i][j] * U[j][k] for j in range(i))) / L[i][i]

        print(f"Step {i + 1}:")
        print("L:", L, "\n", "U:", U, "\n")
    
    return L, U

In [19]:
def verify_lu_decomposition(A, L, U):
    """
    Verify LU decomposition by multiplying L and U and comparing with original matrix.
    
    Parameters:
    A (numpy.ndarray): Original matrix
    L (numpy.ndarray): Lower triangular matrix
    U (numpy.ndarray): Upper triangular matrix
    """
    print("\nLower triangular matrix L:")
    print(L)
    print("\nUpper triangular matrix U:")
    print(U)
    
    # Multiply L and U
    LU = np.dot(L, U)
    print("\nL × U:")
    print(LU)
    
    # Check if LU equals A
    if np.allclose(LU, A):
        print("\nVerification successful: LU = A")
    else:
        print("\nVerification failed: LU ≠ A")

## Kết quả

In [20]:
#Original matrix Ax=B
A = input_matrix('LU_input_A.txt', convert_fractions=False)

print("\nMatrix A:"); output_matrix(A)


Matrix A:
 2.0  3.0 -5.0
 1.0  4.0  2.0
 3.0  7.0  1.0


In [21]:
if A is not None:
    L, U = lu_decomposition(A)
    verify_lu_decomposition(A, L, U)

Step 1:
L: [[2. 0. 0.]
 [1. 0. 0.]
 [3. 0. 0.]] 
 U: [[ 1.   1.5 -2.5]
 [ 0.   1.   0. ]
 [ 0.   0.   1. ]] 

Step 2:
L: [[2.  0.  0. ]
 [1.  2.5 0. ]
 [3.  2.5 0. ]] 
 U: [[ 1.   1.5 -2.5]
 [ 0.   1.   1.8]
 [ 0.   0.   1. ]] 

Step 3:
L: [[2.  0.  0. ]
 [1.  2.5 0. ]
 [3.  2.5 4. ]] 
 U: [[ 1.   1.5 -2.5]
 [ 0.   1.   1.8]
 [ 0.   0.   1. ]] 


Lower triangular matrix L:
[[2.  0.  0. ]
 [1.  2.5 0. ]
 [3.  2.5 4. ]]

Upper triangular matrix U:
[[ 1.   1.5 -2.5]
 [ 0.   1.   1.8]
 [ 0.   0.   1. ]]

L × U:
[[ 2.  3. -5.]
 [ 1.  4.  2.]
 [ 3.  7.  1.]]


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''